In [1]:
import json
import pandas as pd
import pickle
from tworld.prompts import get_functions, get_shared_args, get_system_prompt
from tworld.tworld_funcs import run_and_evaluate

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Configuration
API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

In [3]:
source_dir = 'source'
result_dir = 'result'

In [4]:
# with open('function.json', 'r') as f:
#   FUNCTIONS = json.load(f)

# func_desc = get_functions(FUNCTIONS)

from source import func_description

func_desc = func_description.function_desc

In [5]:
shared_args_file_name = 'shared_arguments.json'
shared_args_file_path = os.path.join(source_dir, shared_args_file_name)
with open(shared_args_file_path, 'r') as f:
    shared_arguments = json.load(f)

args_desc = get_shared_args(shared_arguments)

In [6]:
file_name = "요금 Agent Intent 정의 및 발화 예시.xlsx"
file_path = os.path.join(source_dir, file_name)
plan_list_data = pd.read_excel(file_path, sheet_name='요금제 목록')
plan_list = list(plan_list_data['상품명'].unique())
lineup_list = list(plan_list_data['요금제 라인업, 혜택 라인업'].unique())

In [7]:
from source.examples import examples

system_prompt = get_system_prompt(
    # plan_list=plan_list, 
    # lineup_list=lineup_list, 
    args_desc=args_desc,
    functions_desc=func_desc,
    examples=examples
)
print(system_prompt)

You're a function classifier that needs to identify the appropriate function related to SKTelecom rate plans in order to accurately respond to the user's utterances.  You're actively involved in a three-way conversation with 'user', 'function' and yourself ('assistant').  You must classify the appropriate "function name" and "arguments" according to the user's utterance, and keep the following rules:
    1. "Function name" must be classified only from the lists provided below. You SHOULD NEVER GUESS and CREATE something that is not in the defined list.
    2. Arguments may or may not be required depending on the selected function.
    3. If the selected function has a 'required' field, you must fill in the arguments and send it.
    4. Arguments can be inferred from the user's utterance or the previous conversation.

### Shared Arguments
These argumentsd are shared by multiple functions.
    1. **keywords** (object):
    	- Description: search keywords
    	- Keys:
    		- productName 

In [8]:
# Partially evaluate with false_index(inaccurate functions or arguments)
# false_index_decoded_file_name = 'false_index_decoded.pkl'
# false_index_decoded_file_path = os.path.join(result_dir, false_index_decoded_file_name)
# with open(false_index_decoded_file_path, 'rb') as f:
#     false_index_df = pickle.load(f)
#     target_rows = set(false_index_df.index)

# Partially evaluate with content_filtered(sexuality)
# with open('content_filtered_decoded.pkl', 'rb') as f:
#     content_filtered_df = pickle.load(f)
#     target_rows = set(content_filtered_df.index)

# print(target_rows)


In [9]:
file_name = 'false_index_decoded_abridged_2024-10-23.pkl' # removed AVAILABLE_MOBILE_PLAN, mobile plans and lineups from the system prompt
file_path = os.path.join(result_dir, file_name)
with open(file_path, 'rb') as f:
    df:pd.DataFrame = pickle.load(f)
name_mismatch = df[df['mismatch_args'].isnull()].copy()
target_rows = list(name_mismatch.index)

In [10]:
from datetime import date
from utils import get_aoai_endpoint_in4u

eval_file_name = "(SKT) 평가 결과_new.xlsx"
eval_file_path = os.path.join(source_dir, eval_file_name)
user_query_data = pd.read_excel(eval_file_path)

# Clean up column names by replacing spaces with underscores
user_query_data.columns = [c.replace(' ', '_') for c in user_query_data.columns]

# Define the maximum number of retries
max_retries = 3  # You can change this value as needed

kwargs = {
    'data' : user_query_data,
    'system_prompt' : system_prompt,
    'endpoint': get_aoai_endpoint_in4u(),
    'temparature': 0.0,
    'max_retries' : 5,
    'wait_seconds' : 60,
    'headers' : headers,
    'content_filter_file_name': f'content_filtered_gpt4o_v202408_{date.today()}.pkl',
    'false_index_file_name': f'false_index_gpt4o_v202408_{date.today()}.pkl',
    'today' : date.today(),
    # 'skip_rows': 507,
    'target_rows' : target_rows # target_rows
}

results = run_and_evaluate(**kwargs)
results

row_id = '599': 100%|██████████| 600/600 [01:08<00:00,  8.73it/s]


{'false_index_list': {'390': {'message': {'mismatch_info': "Value mismatch at path 'name': SEARCH_MOBILE_PLAN != check_plan_availability. type(<class 'str'> != <class 'str'>)",
    'true_output': {'name': 'SEARCH_MOBILE_PLAN',
     'arguments': {'keywords': {'productName': '0 청년 다이렉트 69'}}},
    'assistant_output': {'name': 'check_plan_availability',
     'arguments': {'keywords': [{'productName': '0 청년 다이렉트 69'}],
      'requestForMultiline': False}}},
   'user_query': '0 청년 다이렉트 69 요금제 가입 가능해? 혜택도 궁금해'},
  '391': {'message': {'mismatch_info': "Value mismatch at path 'name': SEARCH_MOBILE_PLAN != COMPARE_CHANGE_MY_MOBILE_PLAN. type(<class 'str'> != <class 'str'>)",
    'true_output': {'name': 'SEARCH_MOBILE_PLAN',
     'arguments': {'keywords': {'productName': 'T끼리 55 요금제'}}},
    'assistant_output': {'name': 'COMPARE_CHANGE_MY_MOBILE_PLAN',
     'arguments': {'keywords': {'productName': 'T끼리 55'}}}},
   'user_query': 'T끼리 55 요금제로 변경해도 괜찮을까? 특장점 있으면 말해줘'},
  '392': {'message': {'misma

In [11]:
results_full_file_path = os.path.join(result_dir, f"results_full_{date.today()}.pkl")
with open(results_full_file_path, 'wb') as f:
    pickle.dump(results, f)